In [1]:
import pandas as pd

from src.files_io import download_and_extract_raw_datasets
from src.preprocessing import preprocess_data

In [2]:
download_and_extract_raw_datasets()

Download completed.
Extracting the file...
Extraction completed.
Downloaded tar.gz file removed.
